In [1]:
!pip install --upgrade torch
!pip install --upgrade onnxruntime
!pip install --upgrade numpy

Requirement already up-to-date: torch in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (1.0.1.post2)
Requirement already up-to-date: onnxruntime in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (0.3.0)
Requirement already up-to-date: numpy in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (1.16.2)


In [3]:
import torch
import numpy
import onnxruntime

In [4]:
print(torch.__version__)
print(onnxruntime.__version__)
print(numpy.__version__)

1.0.1.post2
0.3.0
1.16.2


In [5]:
class IdentityNet(torch.nn.Module):
  def __init__(self):
    super(IdentityNet, self).__init__()
    
    in_dim = 1
    hidden_dim = 50
    out_dim = 1
    
    self.linear0 = torch.nn.Linear(in_dim, hidden_dim)
    self.linear1 = torch.nn.Linear(hidden_dim, out_dim)
   
  def forward(self, x):
    x = torch.relu(self.linear0(x))
    x = torch.relu(self.linear1(x))
    
    return x

In [12]:
model = IdentityNet()

In [13]:
lr = 0.001

criterion = torch.nn.SmoothL1Loss(reduction='none')
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [8]:
record_amt = 10

inputs = torch.arange(0,record_amt,dtype=torch.float).view(-1,1)
outputs = torch.arange(0,record_amt,dtype=torch.float).view(-1,1)

In [9]:
inputs = inputs / record_amt
outputs = outputs / record_amt

print(inputs[5], outputs[5])


tensor([0.5000]) tensor([0.5000])


In [10]:
epochs = 100

In [23]:
epoch_loss = 0
for epoch in range(0,epochs):
  
  for t in range(record_amt):
      # Forward pass: Compute predicted y by passing x to the model
      y_pred = model(inputs[t])

      # Compute and print loss
      loss = criterion(y_pred, outputs[t])
      epoch_loss = loss.item()

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

print(epoch_loss, model(inputs[1]).item())

3.282956413386273e-06 0.10008850693702698


In [24]:
y_pred = model(torch.tensor([0.55],dtype=torch.float))
print(y_pred.item())

0.5439932942390442


In [25]:
onnx_input = torch.tensor([0.55],dtype=torch.float)
input_names = [ "input" ]
torch.onnx.export(model, onnx_input, "identity_net.onnx", verbose=True, input_names=input_names)

graph(%input : Float(1)
      %1 : Float(50, 1)
      %2 : Float(50)
      %3 : Float(1, 50)
      %4 : Float(1)) {
  %5 : Float(1!, 50) = onnx::Transpose[perm=[1, 0]](%1), scope: IdentityNet/Linear[linear0]
  %6 : Float(50) = onnx::MatMul(%input, %5), scope: IdentityNet/Linear[linear0]
  %7 : Float(50) = onnx::Add(%6, %2), scope: IdentityNet/Linear[linear0]
  %8 : Float(50) = onnx::Relu(%7), scope: IdentityNet
  %9 : Float(50!, 1!) = onnx::Transpose[perm=[1, 0]](%3), scope: IdentityNet/Linear[linear1]
  %10 : Float(1) = onnx::MatMul(%8, %9), scope: IdentityNet/Linear[linear1]
  %11 : Float(1) = onnx::Add(%10, %4), scope: IdentityNet/Linear[linear1]
  %12 : Float(1) = onnx::Relu(%11), scope: IdentityNet
  return (%12);
}



In [26]:
session = onnxruntime.InferenceSession("identity_net.onnx")

In [27]:
session.get_modelmeta()
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

print(input_name)
print(output_name)

input
12


In [28]:
indata = numpy.array([0.55], dtype=numpy.float32)
results = session.run(None, {input_name: indata})

In [29]:
print(results)

[array([0.5439933], dtype=float32)]
